# Get game list

In [36]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import os.path
import json
import datetime
n=404
def getList(n):
    linklist=[]
    IDlist=[]
    for page in range(n):
        soup = BeautifulSoup(requests.get('https://store.steampowered.com/search/?ignore_preferences=1&category1=998&os=win&filter=globaltopsellers&page=%d'%page).text)
        soups= soup.find_all(href=re.compile(r"https://store.steampowered.com/app/"))
        for i in soups:
            i = i.attrs['href']
            res=re.search('https://store.steampowered.com/app/(\d+)/',i)
            linklist.append(res.group(0))
            IDlist.append(res.group(1))
    return linklist,IDlist
if os.path.isfile('gamelist.csv'): #cache gamelist
    df = pd.read_csv('gamelist.csv')
else:
    linklist,IDlist=getList(n)
    df = pd.DataFrame(list(zip(linklist,IDlist)), columns =['Link', 'ID'])
    df.to_csv('gamelist.csv')

# Get game data

In [35]:
def getPrice(soup):
    originalPrices = soup.select(".game_area_purchase_game .game_purchase_action .game_purchase_price")
    if len(originalPrices)>0:
        for i in range(len(originalPrices)):
            if re.search('\$',originalPrices[i].string):
                originalPrice = float(re.search('\$([\d.]+)',originalPrices[i].string).group(1))
                return originalPrice, originalPrice
            elif re.search('Free',originalPrices[i].string, re.IGNORECASE):
                return 0.,0.
    originalPrices = soup.select(".game_area_purchase_game .game_purchase_action .discount_original_price")
    discountPrices = soup.select(".game_area_purchase_game .game_purchase_action .discount_final_price")
    for i in range(len(originalPrices)):
        if re.search('\$',originalPrices[i].string):
            originalPrice = float(re.search('\$([\d.]+)',originalPrices[i].string).group(1))
            discountPrice = float(re.search('\$([\d.]+)',discountPrices[i].string).group(1))
            return originalPrice, discountPrice
        elif re.search('Free',originalPrices[i].string, re.IGNORECASE):
            return 0.,0.

def getReview(soup):
    try:
        review = str.strip(str(soup.find_all(class_="nonresponsive_hidden responsive_reviewdesc")[-1].contents[0]))
        rating = re.search('(\d+)\% of the ([\d,]+) user', review).group(1)
        rating = int(rating)
        reviewNum = re.search('(\d+)\% of the ([\d,]+) user', review).group(2)
        reviewNum = int(reviewNum.replace(',',''))
        return rating, reviewNum
    except:
        return None, 0

if not os.path.isfile('gameData.json'):
    gameFile = open('gameData.json','w')
    gameData = {}
    gameFile.close()
else:
    gameFile = open('gameData.json','r')
    try:
        gameData = json.load(gameFile)
    except:
        print('empty file')
        gameData = {}
    gameFile.close()
for p in range(len(df)):
    # if p%100==0:
    #     print(p)
    if str(df['ID'][p]) not in gameData.keys():
        soup = BeautifulSoup(requests.get(df['Link'][p]).text)
        name = str(soup.find(class_="apphub_AppName").string)
        try:
            date = str(soup.find(class_="date").string)
        except:
            date = None
        try:
            originalPrice, discountPrice = getPrice(soup)
        except:
            originalPrice, discountPrice = 0.,0.
        rating, reviewNum = getReview(soup)
        description = str.strip(str(soup.find(class_="game_description_snippet").string))
        gameData[str(df['ID'][p])] = [name,date,originalPrice,discountPrice,rating,reviewNum,description]
gameFile = open('gameData.json','w')
gameFile.write(json.dumps(gameData))
gameFile.close()

# Save data into tree